<a href="https://colab.research.google.com/github/SambhaviPD/RandomCoding/blob/master/Exploring_geospatial_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install sodapy

In [68]:
#!/usr/bin/env python

# make sure to install these packages before running:
# pip install pandas
# pip install sodapy

import pandas as pd
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
#client = Socrata("data.sfgov.org", None)

# Example authenticated client (needed for non-public datasets):
# http://dev.socrata.com/foundry/data.winnipeg.ca/gp3k-am4u
# Every Winnipeg Transit bus is equipped with an on-board computer 
# and GPS that logs whether or not it is on time as it leaves a bus stop.
client = Socrata("data.winnipeg.ca",
                  "yourapitokenhere",
                  username="youremailhere",
                  password="yourpasswordhere")

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("gp3k-am4u", limit=100)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)


In [69]:
results_df

,row_id,stop_number,route_number,route_name,route_destination,day_type,scheduled_time,deviation,location,:@computed_region_6rfj_69jf,:@computed_region_38v8_cedi
0,857444741,10355,45,Talbot,Kildonan Place,Saturday,2021-09-11T07:43:39.000,-185,"{'type': 'Point', 'coordinates': [-97.12005859...",12,180
1,857444743,10354,45,Talbot,Kildonan Place,Saturday,2021-09-11T07:44:10.000,-170,"{'type': 'Point', 'coordinates': [-97.11703784...",12,180
2,857444745,10351,45,Talbot,Kildonan Place,Saturday,2021-09-11T07:44:28.000,-163,"{'type': 'Point', 'coordinates': [-97.11540064...",12,180
3,857444747,30101,45,Talbot,Kildonan Place,Saturday,2021-09-11T07:44:53.000,-152,"{'type': 'Point', 'coordinates': [-97.11465630...",4,85
4,857444749,40465,45,Talbot,Kildonan Place,Saturday,2021-09-11T07:46:35.000,-218,"{'type': 'Point', 'coordinates': [-97.11434735...",1,197
...,...,...,...,...,...,...,...,...,...,...,...
95,857444938,40465,45,Talbot,Kildonan Place,Saturday,2021-09-11T09:05:35.000,-169,"{'type': 'Point', 'coordinates': [-97.11434735...",1,197
96,857444940,40467,45,Talbot,Kildonan Place,Saturday,2021-09-11T09:06:19.000,-154,"{'type': 'Point', 'coordinates': [-97.11064201...",1,197
97,857444942,40461,45,Talbot,Kildonan Place,Saturday,2021-09-11T09:06:51.000,-164,"{'type': 'Point', 'coordinates': [-97.10754347...",1,197
98,857444944,40458,45,Talbot,Kildonan Place,Saturday,2021-09-11T09:07:27.000,-156,"{'type': 'Point', 'coordinates': [-97.10407202...",1,197


In [70]:
results_df.columns

Index(['row_id', 'stop_number', 'route_number', 'route_name',
       'route_destination', 'day_type', 'scheduled_time', 'deviation',
       'location', ':@computed_region_6rfj_69jf',
       ':@computed_region_38v8_cedi'],
      dtype='object')

In [71]:
results_df['location']

0     {'type': 'Point', 'coordinates': [-97.12005859...
1     {'type': 'Point', 'coordinates': [-97.11703784...
2     {'type': 'Point', 'coordinates': [-97.11540064...
3     {'type': 'Point', 'coordinates': [-97.11465630...
4     {'type': 'Point', 'coordinates': [-97.11434735...
                            ...                        
95    {'type': 'Point', 'coordinates': [-97.11434735...
96    {'type': 'Point', 'coordinates': [-97.11064201...
97    {'type': 'Point', 'coordinates': [-97.10754347...
98    {'type': 'Point', 'coordinates': [-97.10407202...
99    {'type': 'Point', 'coordinates': [-97.10135948...
Name: location, Length: 100, dtype: object

In [72]:
results_df['latitude'] = [point['coordinates'][0] for point in results_df['location']]
results_df['longitude'] = [point['coordinates'][1] for point in results_df['location']]

In [73]:
results_df['latitude']

0    -97.120059
1    -97.117038
2    -97.115401
3    -97.114656
4    -97.114347
        ...    
95   -97.114347
96   -97.110642
97   -97.107543
98   -97.104072
99   -97.101359
Name: latitude, Length: 100, dtype: float64

In [74]:
results_df['longitude']

0     49.902170
1     49.901779
2     49.902337
3     49.903926
4     49.908634
        ...    
95    49.908634
96    49.907575
97    49.907030
98    49.906407
99    49.905967
Name: longitude, Length: 100, dtype: float64

In [37]:
from geopy.geocoders import Nominatim

In [38]:
# create the locator
geolocator = Nominatim(user_agent="myGeocoder")

In [50]:
# getting the location address
location = geolocator.reverse("49.848606535074, -97.166249398091")

In [51]:
location

Location(Beaumont Station, Southwest Transitway, Beaumont, River Heights–Fort Garry, Winnipeg, Manitoba, R3M 3Y7, Canada, (49.8486899, -97.16583533695267, 0.0))

In [52]:
import folium

In [53]:
# create the map.
map_pickup = folium.Map( location=[49.848606535074,-97.166249398091])

In [75]:
# adding the latitude and longitude points to the map.
results_df.apply(lambda row:folium.CircleMarker(location=[results_df["latitude"], results_df["longitude"]] ).add_to(map_pickup), axis=1)

0     <folium.vector_layers.CircleMarker object at 0...
1     <folium.vector_layers.CircleMarker object at 0...
2     <folium.vector_layers.CircleMarker object at 0...
3     <folium.vector_layers.CircleMarker object at 0...
4     <folium.vector_layers.CircleMarker object at 0...
                            ...                        
95    <folium.vector_layers.CircleMarker object at 0...
96    <folium.vector_layers.CircleMarker object at 0...
97    <folium.vector_layers.CircleMarker object at 0...
98    <folium.vector_layers.CircleMarker object at 0...
99    <folium.vector_layers.CircleMarker object at 0...
Length: 100, dtype: object

In [77]:
# display the map
map_pickup

In [67]:
# optional: save the map.
map_pickup.save('map_pickup.html')